In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt', './data/shakespeare.txt')

In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text = text[100000:1000000]

In [5]:
characters = sorted(set(text))

In [6]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [7]:
SEQ_LENGTH = 48
STEP_SIZE = 3

In [8]:
sentences = []
next_characters = []

In [9]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i + SEQ_LENGTH])
  next_characters.append(text[i + SEQ_LENGTH])

In [10]:
X = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
Y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

In [11]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    X[i, t, char_to_index[character]] = 1
  Y[i, char_to_index[next_characters[i]]] = 1

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
csv_logger = tf.keras.callbacks.CSVLogger('./training_log/log.csv', separator=',', append=True)

model.fit(X, Y, batch_size=256, epochs=5, callbacks=[csv_logger])

Epoch 1/5
1172/1172 [==============================] - 146s 121ms/step - loss: 1.9256
Epoch 2/5
1172/1172 [==============================] - 140s 119ms/step - loss: 1.5771
Epoch 3/5
1172/1172 [==============================] - 146s 124ms/step - loss: 1.4965
Epoch 4/5
1172/1172 [==============================] - 159s 135ms/step - loss: 1.4538
Epoch 5/5
1172/1172 [==============================] - 139s 119ms/step - loss: 1.4259


In [13]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


In [14]:
model = tf.keras.models.load_model('textgenerator.model')

In [15]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [16]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    A = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence):
      A[0, t, char_to_index[character]] = 1
    
    predictions = model.predict(A, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character
  
  return generated

In [17]:
print('-----------------------------0.2-------------------------------')
print(generate_text(256, 0.2))
print('-----------------------------0.4-------------------------------')
print(generate_text(256, 0.4))
print('-----------------------------0.6-------------------------------')
print(generate_text(256, 0.6))
print('-----------------------------0.8-------------------------------')
print(generate_text(256, 0.8))
print('------------------------------1--------------------------------')
print(generate_text(256, 1))

-----------------------------0.2-------------------------------
ll how to term it.

first servingman:
he had so; the say the fearing of the death and the
deed and the hand be state of the head and the strain
to the head and the sould so heart the death,
and the proper the strains and in the blother,
and the heaven of the heart the straight of the head.

clarence:
i 
-----------------------------0.4-------------------------------
e foresaid prunes,--

froth:
ay, so i did indeed the death:
and with his time and the bloth her sense,
which was not so mean the friends and the fracuers
as a man to me to the state of the gremis me
this bear the honour of your following.

corizeang:
i am her state in a king of the state.

duchess:
thy 
-----------------------------0.6-------------------------------
d, early next thursday morn,
the gallant, young the bless her be a prusted,
bear the sensence to the heavy ne'er should were of the souls
of subject for the blort. wherefore a partens.

petrich:
sir